# Cassava Leaf Disease Classification Submission

The notebook where this model is created can be found [here](https://www.kaggle.com/akashsdas/cassava-leaf-disease-classification/) and older submission for models of that notebook can be [here](https://www.kaggle.com/akashsdas/cassava-leaf-disease-classification-old-submission/)

In [ ]:
import numpy as np
import pandas as pd

import cv2

from tensorflow.keras import preprocessing
from tensorflow.keras.models import load_model

The model used here is `model-without-xla-and-mixed-precision.h5` because while using the other model `model.h5`, I was getting the below error as it was trained using `TPU` with `XLA` and `Mixed Precision` enabled.

```bash
InvalidArgumentError: No OpKernel was registered to support Op 'Conv2D' used by {{node sequential/inception_v3/conv2d/Conv2D}} with these attrs: [dilations=[1, 1, 1, 1], T=DT_BFLOAT16, strides=[1, 2, 2, 1], data_format="NHWC", explicit_paddings=[], use_cudnn_on_gpu=true, padding="VALID"]
Registered devices: [CPU, GPU]
Registered kernels:
  device='XLA_CPU_JIT'; T in [DT_FLOAT, DT_DOUBLE, DT_BFLOAT16, DT_HALF]
  device='XLA_GPU_JIT'; T in [DT_FLOAT, DT_DOUBLE, DT_BFLOAT16, DT_HALF]
  device='GPU'; T in [DT_INT32]
  device='GPU'; T in [DT_DOUBLE]
  device='GPU'; T in [DT_FLOAT]
  device='GPU'; T in [DT_HALF]
  device='CPU'; T in [DT_INT32]
  device='CPU'; T in [DT_DOUBLE]
  device='CPU'; T in [DT_FLOAT]
  device='CPU'; T in [DT_HALF]

	 [[sequential/inception_v3/conv2d/Conv2D]] [Op:__inference_predict_function_11991]
```

In [ ]:
# constants

IMAGE_SIZE = [512, 512]

MODEL_DIR = '../input/cassava-leaf-disease-classification-model/model-without-xla-and-mixed-precision.h5'
TEST_DIR = '../input/cassava-leaf-disease-classification/sample_submission.csv'
TEST_IMG_DIR = '../input/cassava-leaf-disease-classification/test_images/'

In [ ]:
# data preparation

test_df = pd.read_csv(TEST_DIR)
test_df['label'] = test_df['label'].astype('str')

test_data_gen = preprocessing.image.ImageDataGenerator(rescale=1./255)
test_gen = test_data_gen.flow_from_dataframe(
    test_df,
    directory=TEST_IMG_DIR,
    x_col='image_id',
    y_col='label',
    target_size=IMAGE_SIZE,
    class_mode='sparse'
)

In [ ]:
# loading the model trained using tpu

model = load_model(MODEL_DIR)
model.summary()

In [ ]:
# prediction

predictions = model.predict(test_gen)
pred_labels = [] # [1, 2, 0, 4, 2, ...]

for pred in predictions:
    pred_labels.append(np.argmax(pred))
    
pred_labels

In [ ]:
# submission

submission = pd.DataFrame({'image_id': test_df.image_id, 'label': pred_labels})
submission.to_csv('submission.csv', index=False) 
submission.head()

---